In [1]:
from collections import defaultdict
from collections import Counter
import MySQLdb
import pandas as pd
import numpy as np
import sys
reload(sys)
sys.setdefaultencoding('utf8')
#import numexpr as ne
#import matplotlib.pyplot as plt
#%matplotlib inline

In [2]:
#Eclipse
conn = MySQLdb.connect(
                     host='localhost', port=3306, user='***', passwd='***', db = 'eclipse2011',
                     charset='utf8')

In [3]:
bug = pd.read_sql("select bug_id, creation_ts, rep_platform, op_sys, assigned_to, reporter, bug_severity, priority, component_id, product_id from bugs where product_id <=5 and bug_status='RESOLVED' and creation_ts<'2007-11' and creation_ts>'2001-10'", conn)

In [4]:
bug.shape

(89075, 10)

In [5]:
for rownum in range(0, bug.shape[0]):
    row = bug.iloc[rownum]
    reporttime = str(row['creation_ts'])
    year = reporttime[0:4]
    month = reporttime[5:7]
    day = reporttime[8:10]
    try:
        bug.ix[rownum, 'YEAR'] = year
        bug.ix[rownum, 'MONTH'] = month
        bug.ix[rownum, 'DAY'] = day
    except Exception:
        pass

In [6]:
bug.to_csv('csmr12/csmr12_eclipse_last.csv',index=False)

In [7]:
bugs_activity = pd.read_sql("select bug_id, bug_when, fieldid, removed, added from bugs_activity where fieldid in (3, 5, 7, 12, 13, 14, 15)", conn)

In [8]:
bugs_activity = bugs_activity[bugs_activity.bug_id.isin(bug.bug_id)]

In [9]:
component_df = pd.read_sql('SELECT name,id FROM components ', conn)
component_df.set_index('name', inplace=True)
component_mapping = component_df['id'].to_dict()

In [10]:
product_df = pd.read_sql('SELECT name,id FROM products ', conn)
product_df.set_index('name', inplace=True)
product_mapping = product_df['id'].to_dict()

In [11]:
bug.set_index('bug_id', inplace=True)
bug.head()

,creation_ts,rep_platform,op_sys,assigned_to,reporter,bug_severity,priority,component_id,product_id,YEAR,MONTH,DAY
bug_id,,,,,,,,,,,,
2,2001-10-10 21:34:47,All,All,25,21,normal,P5,6,1,2001,10,10
3,2001-10-10 21:34:48,All,Windows All,25,57,normal,P5,6,1,2001,10,10
4,2001-10-10 21:34:49,All,All,37,53,normal,P5,6,1,2001,10,10
5,2001-10-10 21:34:50,All,Windows NT,10,51,normal,P3,6,1,2001,10,10
6,2001-10-10 21:34:51,All,All,14,48,normal,P5,6,1,2001,10,10


In [12]:
bugs_activity.sort('bug_when', ascending=False, inplace=True)
for rownum in range(0, bugs_activity.shape[0]):
    row = bugs_activity.iloc[rownum]
    bug_id = row['bug_id']
    field = row['fieldid']
    removed = row['removed']
    try:
        if field == 3:
            bug.ix[bug_id, 'product_id'] = product_mapping[removed]
        elif field == 5:
            bug.ix[bug_id, 'rep_platform'] = removed
        elif field == 7:
            bug.ix[bug_id, 'op_sys'] = removed
        elif field == 12:
            bug.ix[bug_id, 'bug_severity'] = removed
        elif field == 13:
            bug.ix[bug_id, 'priority'] = removed
        elif field == 14:
            bug.ix[bug_id, 'component_id'] = component_mapping[removed]
        elif field == 15:
            bug.ix[bug_id, 'assigned_to'] = removed
    except Exception:
        pass

In [13]:
bug = bug.reset_index()
bug.to_csv('csmr12/csmr12_eclipse_initial.csv',index=False)

In [14]:
#bug_resolvetime = pd.read_sql("select bug_id, max(bug_when) as fixtime from bugs_activity where fieldid=8 and added='RESOLVED' group by bug_id ", conn)
bug_resolvetime = pd.read_sql("select bug_id, delta_ts from bugs ", conn)
bug_resolvetime = pd.merge(bug_resolvetime, bug[['bug_id','creation_ts']], on='bug_id')
bug_resolvetime['interval'] = bug_resolvetime['delta_ts'] - bug_resolvetime['creation_ts']
bug_resolvetime.interval = bug_resolvetime.interval.astype('timedelta64[h]')
bug_resolvetime[['bug_id','interval']].to_csv('csmr12/csmr12_eclipse_label.csv',index=False)

In [1]:
#no filter
import pandas as pd
csmr12_eclipse_initial = pd.read_csv('csmr12/csmr12_eclipse_initial.csv')
csmr12_eclipse_last = pd.read_csv('csmr12/csmr12_eclipse_last.csv')
csmr12_eclipse_label = pd.read_csv('csmr12/csmr12_eclipse_label.csv')
csmr12_eclipse_userid = pd.read_csv('bugs_assignee_init_final.txt', names=['bug_id', 'init', 'final'], sep=';')
csmr12_eclipse_initial.shape

(89075, 13)

In [3]:
#issameproduct = pd.merge(csmr12_eclipse_initial[['bug_id','product_id']],csmr12_eclipse_last[['bug_id','product_id']], on='bug_id')

In [4]:
#issameproduct = issameproduct[issameproduct.product_id_x==issameproduct.product_id_y]
#issameproduct.shape

(83252, 3)

In [5]:
#csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, issameproduct[['bug_id']], on='bug_id')
#csmr12_eclipse_initial.shape

(83252, 13)

In [6]:
#csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, issameproduct[['bug_id']], on='bug_id')
#csmr12_eclipse_label = pd.merge(csmr12_eclipse_label, issameproduct[['bug_id']], on='bug_id')

In [2]:
csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, csmr12_eclipse_userid[['bug_id', 'init']], on='bug_id')
csmr12_eclipse_initial['assigned_to'] = csmr12_eclipse_initial['init']
del csmr12_eclipse_initial['init']
del csmr12_eclipse_initial['creation_ts']
#del csmr12_eclipse_initial['index']
csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, csmr12_eclipse_label, on='bug_id')

In [3]:
median_interval = csmr12_eclipse_initial[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, median_interval, on='product_id')
csmr12_eclipse_initial.loc[csmr12_eclipse_initial.interval<=csmr12_eclipse_initial.med, 'label'] = 'fast'
csmr12_eclipse_initial.loc[csmr12_eclipse_initial.interval>csmr12_eclipse_initial.med, 'label'] = 'slow'
csmr12_eclipse_initial[csmr12_eclipse_initial.product_id==1].to_csv('csmr12/csmr12_eclipse_init_Platform_nofilter.csv',index=False)
csmr12_eclipse_initial[csmr12_eclipse_initial.product_id==2].to_csv('csmr12/csmr12_eclipse_init_PDE_nofilter.csv',index=False)
csmr12_eclipse_initial[csmr12_eclipse_initial.product_id==3].to_csv('csmr12/csmr12_eclipse_init_JDT_nofilter.csv',index=False)
csmr12_eclipse_initial[csmr12_eclipse_initial.product_id==4].to_csv('csmr12/csmr12_eclipse_init_CDT_nofilter.csv',index=False)
csmr12_eclipse_initial[csmr12_eclipse_initial.product_id==5].to_csv('csmr12/csmr12_eclipse_init_GEF_nofilter.csv',index=False)

csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, csmr12_eclipse_userid[['bug_id', 'final']], on='bug_id')
csmr12_eclipse_last['assigned_to'] = csmr12_eclipse_last['final']
del csmr12_eclipse_last['final']
del csmr12_eclipse_last['creation_ts']
csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, csmr12_eclipse_label, on='bug_id')
median_interval = csmr12_eclipse_last[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, median_interval, on='product_id')
csmr12_eclipse_last.loc[csmr12_eclipse_last.interval<=csmr12_eclipse_last.med, 'label'] = 'fast'
csmr12_eclipse_last.loc[csmr12_eclipse_last.interval>csmr12_eclipse_last.med, 'label'] = 'slow'
csmr12_eclipse_last[csmr12_eclipse_last.product_id==1].to_csv('csmr12/csmr12_eclipse_last_Platform_nofilter.csv',index=False)
csmr12_eclipse_last[csmr12_eclipse_last.product_id==2].to_csv('csmr12/csmr12_eclipse_last_PDE_nofilter.csv',index=False)
csmr12_eclipse_last[csmr12_eclipse_last.product_id==3].to_csv('csmr12/csmr12_eclipse_last_JDT_nofilter.csv',index=False)
csmr12_eclipse_last[csmr12_eclipse_last.product_id==4].to_csv('csmr12/csmr12_eclipse_last_CDT_nofilter.csv',index=False)
csmr12_eclipse_last[csmr12_eclipse_last.product_id==5].to_csv('csmr12/csmr12_eclipse_last_GEF_nofilter.csv',index=False)

In [1]:
#filter
import pandas as pd
from __future__ import division
from collections import defaultdict
import random

csmr12_eclipse_initial = pd.read_csv('csmr12/csmr12_eclipse_initial.csv')
csmr12_eclipse_last = pd.read_csv('csmr12/csmr12_eclipse_last.csv')
csmr12_eclipse_label = pd.read_csv('csmr12/csmr12_eclipse_label.csv')
csmr12_eclipse_userid = pd.read_csv('bugs_assignee_init_final.txt', names=['bug_id', 'init', 'final'], sep=';')

In [2]:
#issameproduct = pd.merge(csmr12_eclipse_initial[['bug_id','product_id']],csmr12_eclipse_last[['bug_id','product_id']], on='bug_id')
#issameproduct = issameproduct[issameproduct.product_id_x==issameproduct.product_id_y]
#csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, issameproduct[['bug_id']], on='bug_id')
#csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, issameproduct[['bug_id']], on='bug_id')
#csmr12_eclipse_label = pd.merge(csmr12_eclipse_label, issameproduct[['bug_id']], on='bug_id')

In [2]:
csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, csmr12_eclipse_userid[['bug_id', 'init']], on='bug_id')
csmr12_eclipse_initial['assigned_to'] = csmr12_eclipse_initial['init']
del csmr12_eclipse_initial['init']
del csmr12_eclipse_initial['creation_ts']
#del csmr12_eclipse_initial['index']
csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, csmr12_eclipse_label, on='bug_id')
threshold_interval = csmr12_eclipse_initial[['product_id', 'interval']].groupby('product_id').quantile(0.125).reset_index()
threshold_interval['threshold'] = threshold_interval['interval']
del threshold_interval['interval']
csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, threshold_interval, on='product_id')
csmr12_eclipse_initial = csmr12_eclipse_initial[csmr12_eclipse_initial.interval>csmr12_eclipse_initial.threshold]

median_interval = csmr12_eclipse_initial[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_eclipse_initial = pd.merge(csmr12_eclipse_initial, median_interval, on='product_id')
csmr12_eclipse_initial.loc[csmr12_eclipse_initial.interval<=csmr12_eclipse_initial.med, 'label'] = 'fast'
csmr12_eclipse_initial.loc[csmr12_eclipse_initial.interval>csmr12_eclipse_initial.med, 'label'] = 'slow'

In [3]:
csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, csmr12_eclipse_userid[['bug_id', 'final']], on='bug_id')
csmr12_eclipse_last['assigned_to'] = csmr12_eclipse_last['final']
del csmr12_eclipse_last['final']
del csmr12_eclipse_last['creation_ts']
csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, csmr12_eclipse_label, on='bug_id')
threshold_interval = csmr12_eclipse_last[['product_id', 'interval']].groupby('product_id').quantile(0.125).reset_index()
threshold_interval['threshold'] = threshold_interval['interval']
del threshold_interval['interval']
csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, threshold_interval, on='product_id')
csmr12_eclipse_last = csmr12_eclipse_last[csmr12_eclipse_last.interval>csmr12_eclipse_last.threshold]

median_interval = csmr12_eclipse_last[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_eclipse_last = pd.merge(csmr12_eclipse_last, median_interval, on='product_id')
csmr12_eclipse_last.loc[csmr12_eclipse_last.interval<=csmr12_eclipse_last.med, 'label'] = 'fast'
csmr12_eclipse_last.loc[csmr12_eclipse_last.interval>csmr12_eclipse_last.med, 'label'] = 'slow'

In [4]:
csmr12_eclipse_initial[csmr12_eclipse_initial.product_id==1].to_csv('csmr12/csmr12_eclipse_init_Platform_filter.csv',index=False)
csmr12_eclipse_initial[csmr12_eclipse_initial.product_id==2].to_csv('csmr12/csmr12_eclipse_init_PDE_filter.csv',index=False)
csmr12_eclipse_initial[csmr12_eclipse_initial.product_id==3].to_csv('csmr12/csmr12_eclipse_init_JDT_filter.csv',index=False)
csmr12_eclipse_initial[csmr12_eclipse_initial.product_id==4].to_csv('csmr12/csmr12_eclipse_init_CDT_filter.csv',index=False)
csmr12_eclipse_initial[csmr12_eclipse_initial.product_id==5].to_csv('csmr12/csmr12_eclipse_init_GEF_filter.csv',index=False)

csmr12_eclipse_last[csmr12_eclipse_last.product_id==1].to_csv('csmr12/csmr12_eclipse_last_Platform_filter.csv',index=False)
csmr12_eclipse_last[csmr12_eclipse_last.product_id==2].to_csv('csmr12/csmr12_eclipse_last_PDE_filter.csv',index=False)
csmr12_eclipse_last[csmr12_eclipse_last.product_id==3].to_csv('csmr12/csmr12_eclipse_last_JDT_filter.csv',index=False)
csmr12_eclipse_last[csmr12_eclipse_last.product_id==4].to_csv('csmr12/csmr12_eclipse_last_CDT_filter.csv',index=False)
csmr12_eclipse_last[csmr12_eclipse_last.product_id==5].to_csv('csmr12/csmr12_eclipse_last_GEF_filter.csv',index=False)

In [1]:
from collections import defaultdict
from collections import Counter
import MySQLdb
import pandas as pd
import numpy as np
import sys
reload(sys)
sys.setdefaultencoding('utf8')
#import numexpr as ne
#import matplotlib.pyplot as plt
#%matplotlib inline

#Mozilla
conn = MySQLdb.connect(
                     host='localhost', port=3306, user='***', passwd='***', db = 'mozilla2016',
                     charset='utf8')

bug = pd.read_sql("select bug_id, creation_ts, rep_platform, op_sys, assigned_to, reporter, bug_severity, priority, component_id, product_id from bugs where product_id in (1, 19, 21, 23, 25) and bug_status='RESOLVED' and creation_ts<'2008-08'", conn)

for rownum in range(0, bug.shape[0]):
    row = bug.iloc[rownum]
    reporttime = str(row['creation_ts'])
    year = reporttime[0:4]
    month = reporttime[5:7]
    day = reporttime[8:10]
    bug.ix[rownum, 'YEAR'] = year
    bug.ix[rownum, 'MONTH'] = month
    bug.ix[rownum, 'DAY'] = day

In [2]:
bug.to_csv('csmr12/csmr12_mozilla_last.csv',index=False)
bugs_activity = pd.read_sql("select bug_id, bug_when, fieldid, removed, added from bugs_activity where fieldid in (25, 27, 28, 31, 32, 33, 34)", conn)
bugs_activity = bugs_activity[bugs_activity.bug_id.isin(bug.bug_id)]

component_df = pd.read_sql('SELECT  name,id FROM components ', conn)
component_df.set_index('name', inplace=True)
component_mapping = component_df['id'].to_dict()

product_df = pd.read_sql('SELECT name,id FROM products ', conn)
product_df.set_index('name', inplace=True)
product_mapping = product_df['id'].to_dict()

user_df = pd.read_sql('SELECT  login_name, userid FROM profiles ', conn)
user_df.set_index('login_name', inplace=True)
user_mapping = user_df['userid'].to_dict()

bug.set_index('bug_id', inplace=True)
bugs_activity.sort('bug_when', ascending=False, inplace=True)

In [3]:
for rownum in range(0, bugs_activity.shape[0]):
    row = bugs_activity.iloc[rownum]
    bug_id = row['bug_id']
    field = row['fieldid']
    removed = row['removed']
    try:
        if field == 25:
            bug.ix[bug_id, 'product_id'] = product_mapping[removed]
        elif field == 27:
            bug.ix[bug_id, 'rep_platform'] = removed
        elif field == 28:
            bug.ix[bug_id, 'op_sys'] = removed
        elif field == 31:
            bug.ix[bug_id, 'bug_severity'] = removed
        elif field == 32:
            bug.ix[bug_id, 'priority'] = removed
        elif field == 33:
            bug.ix[bug_id, 'component_id'] = component_mapping[removed]
        elif field == 34:
            bug.ix[bug_id, 'assigned_to'] = user_mapping[removed]
    except Exception:
        pass

In [4]:
    
bug = bug.reset_index()
bug.to_csv('csmr12/csmr12_mozilla_initial.csv',index=False)


bug_resolvetime = pd.read_sql("select bug_id, delta_ts from bugs ", conn)
bug_resolvetime = pd.merge(bug_resolvetime, bug[['bug_id','creation_ts']], on='bug_id')
bug_resolvetime['interval'] = bug_resolvetime['delta_ts'] - bug_resolvetime['creation_ts']
bug_resolvetime.interval = bug_resolvetime.interval.astype('timedelta64[h]')
bug_resolvetime[['bug_id','interval']].to_csv('csmr12/csmr12_mozilla_label.csv',index=False)

In [1]:
#no filter
import pandas as pd
csmr12_mozilla_initial = pd.read_csv('csmr12/csmr12_mozilla_initial.csv')
csmr12_mozilla_last = pd.read_csv('csmr12/csmr12_mozilla_last.csv')
csmr12_mozilla_label = pd.read_csv('csmr12/csmr12_mozilla_label.csv')

#issameproduct = pd.merge(csmr12_mozilla_initial[['bug_id','product_id']],csmr12_mozilla_last[['bug_id','product_id']], on='bug_id')
#issameproduct = issameproduct[issameproduct.product_id_x==issameproduct.product_id_y]
#csmr12_mozilla_initial = pd.merge(csmr12_mozilla_initial, issameproduct[['bug_id']], on='bug_id')
#csmr12_mozilla_last = pd.merge(csmr12_mozilla_last, issameproduct[['bug_id']], on='bug_id')
#csmr12_mozilla_label = pd.merge(csmr12_mozilla_label, issameproduct[['bug_id']], on='bug_id')

del csmr12_mozilla_initial['creation_ts']
csmr12_mozilla_initial = pd.merge(csmr12_mozilla_initial, csmr12_mozilla_label, on='bug_id')

median_interval = csmr12_mozilla_initial[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_mozilla_initial = pd.merge(csmr12_mozilla_initial, median_interval, on='product_id')
csmr12_mozilla_initial.loc[csmr12_mozilla_initial.interval<=csmr12_mozilla_initial.med, 'label'] = 'fast'
csmr12_mozilla_initial.loc[csmr12_mozilla_initial.interval>csmr12_mozilla_initial.med, 'label'] = 'slow'

csmr12_mozilla_initial[csmr12_mozilla_initial.product_id==1].to_csv('csmr12/csmr12_mozilla_init_Core_nofilter.csv',index=False)
csmr12_mozilla_initial[csmr12_mozilla_initial.product_id==19].to_csv('csmr12/csmr12_mozilla_init_Bugzilla_nofilter.csv',index=False)
csmr12_mozilla_initial[csmr12_mozilla_initial.product_id==21].to_csv('csmr12/csmr12_mozilla_init_Firefox_nofilter.csv',index=False)
csmr12_mozilla_initial[csmr12_mozilla_initial.product_id==23].to_csv('csmr12/csmr12_mozilla_init_Thunderbird_nofilter.csv',index=False)
csmr12_mozilla_initial[csmr12_mozilla_initial.product_id==25].to_csv('csmr12/csmr12_mozilla_init_Seamonkey_nofilter.csv',index=False)



del csmr12_mozilla_last['creation_ts']
csmr12_mozilla_last = pd.merge(csmr12_mozilla_last, csmr12_mozilla_label, on='bug_id')

median_interval = csmr12_mozilla_last[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_mozilla_last = pd.merge(csmr12_mozilla_last, median_interval, on='product_id')
csmr12_mozilla_last.loc[csmr12_mozilla_last.interval<=csmr12_mozilla_last.med, 'label'] = 'fast'
csmr12_mozilla_last.loc[csmr12_mozilla_last.interval>csmr12_mozilla_last.med, 'label'] = 'slow'

csmr12_mozilla_last[csmr12_mozilla_last.product_id==1].to_csv('csmr12/csmr12_mozilla_last_Core_nofilter.csv',index=False)
csmr12_mozilla_last[csmr12_mozilla_last.product_id==19].to_csv('csmr12/csmr12_mozilla_last_Bugzilla_nofilter.csv',index=False)
csmr12_mozilla_last[csmr12_mozilla_last.product_id==21].to_csv('csmr12/csmr12_mozilla_last_Firefox_nofilter.csv',index=False)
csmr12_mozilla_last[csmr12_mozilla_last.product_id==23].to_csv('csmr12/csmr12_mozilla_last_Thunderbird_nofilter.csv',index=False)
csmr12_mozilla_last[csmr12_mozilla_last.product_id==25].to_csv('csmr12/csmr12_mozilla_last_Seamonkey_nofilter.csv',index=False)

In [8]:
issameproduct.shape

(167894, 3)

In [1]:
#filter
import pandas as pd

csmr12_mozilla_initial = pd.read_csv('csmr12/csmr12_mozilla_initial.csv')
csmr12_mozilla_last = pd.read_csv('csmr12/csmr12_mozilla_last.csv')
csmr12_mozilla_label = pd.read_csv('csmr12/csmr12_mozilla_label.csv')


#issameproduct = pd.merge(csmr12_mozilla_initial[['bug_id','product_id']],csmr12_mozilla_last[['bug_id','product_id']], on='bug_id')
#issameproduct = issameproduct[issameproduct.product_id_x==issameproduct.product_id_y]
#csmr12_mozilla_initial = pd.merge(csmr12_mozilla_initial, issameproduct[['bug_id']], on='bug_id')
#csmr12_mozilla_last = pd.merge(csmr12_mozilla_last, issameproduct[['bug_id']], on='bug_id')
#csmr12_mozilla_label = pd.merge(csmr12_mozilla_label, issameproduct[['bug_id']], on='bug_id')


del csmr12_mozilla_initial['creation_ts']
csmr12_mozilla_initial = pd.merge(csmr12_mozilla_initial, csmr12_mozilla_label, on='bug_id')
threshold_interval = csmr12_mozilla_initial[['product_id', 'interval']].groupby('product_id').quantile(0.125).reset_index()
threshold_interval['threshold'] = threshold_interval['interval']
del threshold_interval['interval']
csmr12_mozilla_initial = pd.merge(csmr12_mozilla_initial, threshold_interval, on='product_id')
csmr12_mozilla_initial = csmr12_mozilla_initial[csmr12_mozilla_initial.interval>csmr12_mozilla_initial.threshold]

median_interval = csmr12_mozilla_initial[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_mozilla_initial = pd.merge(csmr12_mozilla_initial, median_interval, on='product_id')
csmr12_mozilla_initial.loc[csmr12_mozilla_initial.interval<=csmr12_mozilla_initial.med, 'label'] = 'fast'
csmr12_mozilla_initial.loc[csmr12_mozilla_initial.interval>csmr12_mozilla_initial.med, 'label'] = 'slow'


del csmr12_mozilla_last['creation_ts']
csmr12_mozilla_last = pd.merge(csmr12_mozilla_last, csmr12_mozilla_label, on='bug_id')
threshold_interval = csmr12_mozilla_last[['product_id', 'interval']].groupby('product_id').quantile(0.125).reset_index()
threshold_interval['threshold'] = threshold_interval['interval']
del threshold_interval['interval']
csmr12_mozilla_last = pd.merge(csmr12_mozilla_last, threshold_interval, on='product_id')
csmr12_mozilla_last = csmr12_mozilla_last[csmr12_mozilla_last.interval>csmr12_mozilla_last.threshold]

median_interval = csmr12_mozilla_last[['product_id', 'interval']].groupby('product_id').median().reset_index()
median_interval['med'] = median_interval['interval']
del median_interval['interval']
csmr12_mozilla_last = pd.merge(csmr12_mozilla_last, median_interval, on='product_id')
csmr12_mozilla_last.loc[csmr12_mozilla_last.interval<=csmr12_mozilla_last.med, 'label'] = 'fast'
csmr12_mozilla_last.loc[csmr12_mozilla_last.interval>csmr12_mozilla_last.med, 'label'] = 'slow'

In [2]:
csmr12_mozilla_initial[csmr12_mozilla_initial.product_id==1].to_csv('csmr12/csmr12_mozilla_init_Core_filter.csv',index=False)
csmr12_mozilla_initial[csmr12_mozilla_initial.product_id==19].to_csv('csmr12/csmr12_mozilla_init_Bugzilla_filter.csv',index=False)
csmr12_mozilla_initial[csmr12_mozilla_initial.product_id==21].to_csv('csmr12/csmr12_mozilla_init_Firefox_filter.csv',index=False)
csmr12_mozilla_initial[csmr12_mozilla_initial.product_id==23].to_csv('csmr12/csmr12_mozilla_init_Thunderbird_filter.csv',index=False)
csmr12_mozilla_initial[csmr12_mozilla_initial.product_id==25].to_csv('csmr12/csmr12_mozilla_init_Seamonkey_filter.csv',index=False)

csmr12_mozilla_last[csmr12_mozilla_last.product_id==1].to_csv('csmr12/csmr12_mozilla_last_Core_filter.csv',index=False)
csmr12_mozilla_last[csmr12_mozilla_last.product_id==19].to_csv('csmr12/csmr12_mozilla_last_Bugzilla_filter.csv',index=False)
csmr12_mozilla_last[csmr12_mozilla_last.product_id==21].to_csv('csmr12/csmr12_mozilla_last_Firefox_filter.csv',index=False)
csmr12_mozilla_last[csmr12_mozilla_last.product_id==23].to_csv('csmr12/csmr12_mozilla_last_Thunderbird_filter.csv',index=False)
csmr12_mozilla_last[csmr12_mozilla_last.product_id==25].to_csv('csmr12/csmr12_mozilla_last_Seamonkey_filter.csv',index=False)

In [3]:
import os
import pandas as pd

for filename in os.listdir(r'csmr12'):
    if filename.endswith('_filter.csv'):
        tmp = pd.read_csv('csmr12/'+filename)
        del tmp['product_id']
        del tmp['bug_id']
        del tmp['interval']
        del tmp['med']
        del tmp['threshold']
        tmp.to_csv('csmr12/'+filename,index=False)
    elif filename.endswith('_nofilter.csv'):
        tmp = pd.read_csv('csmr12/'+filename)
        del tmp['product_id']
        del tmp['bug_id']
        del tmp['interval']
        del tmp['med']
        tmp.to_csv('csmr12/'+filename,index=False)